# Repertoire

In [1]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [2]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [3]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [4]:
%%capture
pip install -r requirements.txt

In [5]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np

# FUNCTIONS

In [6]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Discogs data

## Load 

In [7]:
baseline = pd.read_pickle(REP_INTERMED+ "baseline.sav")

## Vectorizer :  Keep TF-IDF strategy 

In [8]:
start_time = datetime.now()

create_vectorizer_and_vectorized_db(baseline, analyzer = 'word',ngram_range=(1,1))

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# Duration: 02 min:15s

**** creation du vectorizer


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


nb features: 1478896
**** sauvegarde des fichiers en pickle pour la prochaine fois
Duration: 0:01:35.180699


In [9]:
analyzer = 'word'
ngram_range = (1,1)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)

# Query Data

In [10]:
df_no_match = pd.read_pickle(REP_INTERMED +'base_cdandlp.sav').reset_index(drop=True)

df_no_match['text'] = df_no_match['artiste'] +  ' ' + df_no_match['titre']

df_sample = df_no_match.sample(100, random_state=20).reset_index(drop=True)

del df_no_match


# KNN unsupervised (sklearn api)

## Training with Manahattan

In [11]:
suffix_gram = re.sub(r"[(,)]", "", str(ngram_range)).replace(' ','_')

path = REP_INTERMED + analyzer + '_' + suffix_gram+  '_results_sim_manhattan.pkl'

nbrs = NearestNeighbors(n_neighbors=100, metric="manhattan",algorithm='brute', n_jobs=-1).fit(X_train)

start_time = datetime.now()

result = [ nbrs.kneighbors(text_to_vect(df_sample.iloc[i,3], vectorizer)) for i in range (len(df_sample)) ]

f = open(path, 'wb')

pickle.dump(result, f) 

f.close()

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

Duration: 0:03:28.990082


# Customiz Output

In [12]:
with open(path, 'rb') as file:
      result = pickle.load(file)

In [13]:
temp = [ customize_output(i,results= result, df_baseline=baseline.copy(), sample=df_sample) for i in range(len(df_sample)) ]

In [14]:
pd.concat(temp,axis=0).\
    rename(columns={'master_id':'id_text'}).\
    to_excel(REP_INTERMED + analyzer + '_' + suffix_gram +'_neighbors_Manhattan.xlsx', index= False)

In [15]:
pd.concat(temp,axis=0)

,id_query,master_id,artist,title,distance,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,114852181,114852181,tiffany,i think we're alone now,-1.0,0,1,1,1,1,1,1
1,114852181,314885,Tiffany,I Think We’Re Alone Now,0.0,1,1.0,1.0,1.0,1.0,1.0,1.0
2,114852181,"314885,74319",Tiffany,I Think We'Re Alone Now,0.0,1,1.0,1.0,1.0,1.0,1.0,1.0
3,114852181,0,Annakiya,I Think We'Re Alone Now,0.816497,3,0.852459,0.773587,0.742857,0.866667,0.0,0.83
4,114852181,0,I Think We'Re Alone Now,Unwritten,0.816497,3,0.709677,0.711528,0.722222,0.866667,0.09375,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...
96,119859709,0,D.C.O.,D.C.O.,2.44949,1,0.25,0.510017,0.191489,0.818182,0.0,0.27
97,119859709,0,C/R/I/T/I/C/S,C/R/I/T/I/C/S,2.44949,1,0.361111,0.474074,0.285714,0.592593,0.044444,0.41
98,119859709,0,C H R Y S A M E R E,C H R Y S A M E R E,2.44949,1,0.285714,0.490883,0.3125,0.512821,0.044444,0.31
99,119859709,0,E.C.P.,E.C.P.,2.44949,1,0.214286,0.502068,0.166667,0.727273,0.0,0.36
